In [24]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [197]:
# 데이터 불러오기
unsold_house = pd.read_excel('./1_데이터 재수집/규모별_미분양현황_20230218162415.xlsx', index_col = 0)
apt_rent = pd.read_excel('./1_데이터 재수집/아파트전세가격지수_시도_시_군_구__20230218162737.xlsx', index_col = 0)
birth = pd.read_excel('./1_데이터 재수집/출생아수_시도_시_군_구__20230218162012.xlsx', index_col = 0)
unemployment_rate = pd.read_excel('./1_데이터 재수집/행정구역_시도__성별_실업률_20230218162056.xlsx', index_col = 0)

### 미분양 현황

In [198]:
# 필요없는 열 삭제
unsold_house.drop(['시도(1)'], inplace = True)

# index, col name 삭제 및 전치
unsold_house.index.name = '시점'
unsold_house.columns.name = None
unsold_house = unsold_house.T

In [199]:
# 연, 월의 데이터 구조 변환 : 2006.01 --> 2006-01-01로 진행

# index를 열로 추가
unsold_house['시점'] = unsold_house.index

# 열 이름 변경
columns = ['전국', '서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', 
           '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도', '시점']
unsold_house = pd.DataFrame(data = unsold_house.values, columns = columns)
unsold_house

# 데이터프레임 형태에서 데이터 구조 통일화 : yyyy.1만 --> yyyy.10으로 변경
unsold_house['시점'] =  unsold_house['시점'].apply(lambda r : r + '0' if len(r) == 6 else r)
unsold_house.set_index('시점', inplace = True)

# 시점 열을 index로 만들어 datetime 형태 진행
unsold_house.index = pd.to_datetime(unsold_house.index, format = '%Y-%m')
unsold_house['세종특별자치시'].replace('-', 0, inplace = True)
unsold_house = unsold_house.apply(pd.to_numeric)
unsold_house.columns.name = None
unsold_house.to_excel('2007-2022_미분양현황.xlsx')

In [206]:
# 아파트(전세)가격지수
apt_rent.index.name = '시점'
apt_rent.columns.name = None
apt_rent = apt_rent.T

apt_rent.index = pd.to_datetime(apt_rent.index, format = '%Y-%m')
apt_rent['세종특별자치시'].replace('-', 0, inplace = True)
apt_rent = apt_rent.apply(pd.to_numeric)
apt_rent.columns.name = None

apt_rent.to_excel('2003-2022_아파트전세_가격지수.xlsx')

In [208]:
# 총 출생아수
birth.index.name = '시점'
birth.columns.name = None
birth = birth.T

birth.index = pd.to_datetime(birth.index, format = '%Y-%m')
birth['세종특별자치시'].replace('-', 0, inplace = True)
birth = birth.apply(pd.to_numeric)
birth.columns.name = None

birth.to_excel('1997-2021_총_출생아수.xlsx')

In [209]:
birth

,전국,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도
1997-01-01,63268,13205,4201,3345,3646,2166,2003,1754,0,13440,1794,2075,2259,2579,2685,3362,4020,734
1997-02-01,58144,12239,3910,2994,3193,1910,1869,1569,0,12334,1648,1798,2082,2261,2547,3257,3796,737
1997-03-01,62160,13358,4304,3337,3466,2013,1894,1771,0,13058,1831,1988,2205,2364,2457,3343,4021,750
1997-04-01,56949,12146,3872,3025,3189,1695,1666,1569,0,12101,1670,1750,2063,2203,2348,3183,3800,669
1997-05-01,55270,11637,3826,2855,3074,1799,1765,1479,0,11644,1650,1734,1966,2085,2290,3024,3716,726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-01,22282,3858,1302,950,1272,671,593,527,290,6473,649,747,943,610,738,1065,1322,272
2021-09-01,21905,3753,1238,893,1320,707,626,513,305,6347,594,649,949,628,708,1001,1334,340
2021-10-01,20749,3533,1183,890,1240,604,592,505,319,6105,618,668,815,588,632,889,1238,330
2021-11-01,19829,3513,1043,792,1189,661,560,451,263,5766,587,631,840,543,640,961,1135,254
